# 0. Multi-Armed Bandit (MAB) 개념 정리

- Multi-Armed Bandit(MAB)은 여러 개의 arm(레버)을 가진 슬롯머신 중 어떤 arm을 선택해야 보상이 가장 큰지를 학습하는 알고리즘임
- 실제 슬롯머신 문제에서 유래한 개념으로, exploration과 exploitation 사이의 trade-off를 다룸

- 예시: 여러 대의 슬롯머신이 있을 때 어떤 머신이 가장 높은 보상을 줄지 모르는 상황
  - 매번 가장 좋아 보이는 arm만 당기면 학습이 느림 (exploitation 중심)
  - 매번 무작위로 선택하면 성능이 낮음 (exploration 중심)
  → 두 전략을 적절히 섞는 것이 핵심

- 이 개념은 추천 시스템에도 적용 가능함
  - 어떤 아이템을 유저에게 추천해야 할지 모를 때, 추천 후 피드백(클릭 여부)을 바탕으로 학습 가능

## 대표적인 MAB 전략

- Greedy: 현재까지 가장 높은 평균 보상을 가진 arm 선택
- 𝜖-greedy: 확률 𝜖으로 무작위 탐색, 1−𝜖 확률로 현재 최적 arm 선택
- Softmax: 각 arm의 예상 보상에 비례해 확률적으로 선택하며, temperature 파라미터로 탐색 정도를 조절

- UCB (Upper Confidence Bound): 기대 보상 + 신뢰 구간을 더한 기준으로 arm 선택


---

#. Contextual Bandit 개념 정리

- Contextual Bandit은 MAB의 확장 구조로, 각 arm 선택 시 상황에 따른 문맥(context) 정보를 활용함
- 예: 사용자의 특성(성별, 나이 등)과 아이템의 정보(카테고리, 노출 시간 등)를 반영해 arm 선택

- 매 trial마다 사용자와 아이템 상황이 바뀌므로, 고정된 평균 보상만으로는 최적 선택이 어려움
- 따라서 각 상황에 맞는 arm을 선택하면서 클릭 피드백을 통해 선택 전략을 점진적으로 개선함

---

# 이 논문이 다루는 문제

- 실시간 웹/모바일 환경에서의 개인화 추천 문제를 해결하고자 함
- 특히 뉴스 기사 추천처럼 item pool과 user preference가 계속 변하는 상황을 고려함
- 주요 목표:
  - 각 사용자 방문 시점에 가장 적절한 아이템(기사)을 추천
  - 장기적으로 사용자 클릭 수의 총합을 최대화

---

# 기존 방식의 한계

1. 콘텐츠 풀의 변화가 잦아 정적인 추천 방식이 적합하지 않음
2. 시간에 따라 콘텐츠 인기도가 달라짐
3. 방문자의 많은 수가 신규 사용자로, 과거 클릭 이력이 없음 (cold-start 문제)

→ 협업 필터링이나 콘텐츠 기반 추천만으로는 이러한 상황에 대응하기 어려움

---

# 논문의 접근 방식: Contextual Bandit 활용

- 사용자와 기사에 대한 문맥 정보를 기반으로 추천 수행
- 클릭 피드백을 바탕으로 추천 전략을 실시간 업데이트
- exploration과 exploitation을 동시에 고려하는 구조를 설계
- 최종 목표는 전체 사용자에 대한 총 클릭 수(누적 보상)를 최대화하는 것

- 이러한 구조를 기반으로, 논문은 Contextual Bandit 문제에 적합한 선택 전략인 LinUCB 알고리즘을 설계함


#  Chapter 1. Introduction

- 논문은 웹(모바일) 환경에서의 개인화 추천 문제를 다룸  
- 유저 방문 시점에 가장 적절한 아이템(뉴스 등)을 추천하는 것이 목적임  
- 뉴스 추천처럼 item pool과 user preference가 시간이 지남에 따라 변하는 환경을 가정함  
- 기존 협업 필터링은 item/user가 고정된다고 가정하므로 이런 환경에 적합하지 않음  
- Bandit 기반 추천은 추천 후 유저 반응(클릭 등)을 통해 피드백을 받고, 추천 전략을 점차 개선함  
- 특히 contextual bandit은 유저의 특징(context)을 반영해 개인화된 추천이 가능함  
- 변화가 많은 실시간 환경에서 더 유효한 접근법임  
- 논문은 이러한 구조에 기반해 알고리즘(LinUCB)을 제안하고, 실제 대규모 로그(Yahoo!)를 통해 성능을 검증함

# Chapter 2. Formulation & Related Work

## < 2.1 A Multi-armed Bandit Formulation >

- 개인화 뉴스 추천 문제는 contextual bandit 구조로 모델링할 수 있음
- 각 trial t에서 알고리즘은 다음 과정을 반복함:
  1. 사용자 u_t와 arm 집합 A_t 관찰, 각 arm a는 feature vector \( x_{t,a} \)를 가짐
  2. 하나의 arm \( a_t \)를 선택하고 보상 \( r_{t,a_t} \)를 받음
  3. 선택 결과를 바탕으로 전략 업데이트

- 특징: 선택하지 않은 arm에 대한 보상은 알 수 없음 (partial feedback 구조)

- 학습 목표는 총 보상 \( \sum_{t=1}^T r_{t,a_t} \)을 최대화하거나  
  regret \( R_A(T) \)을 최소화하는 것임

- context-free bandit 구조에서는:
  - arm set A_t는 고정됨
  - 사용자 feature도 매 trial마다 동일함
  → 유저 context를 고려하지 않으므로 개인화 추천이 불가능함

- contextual bandit 구조에서는:
  - arm set과 사용자 context가 매 trial마다 달라짐
  → 문맥 정보에 기반한 개인화 추천이 가능함

- 두 구조의 차이는 다음 두 가지임:
  1. arm pool의 변화 가능성
  2. 사용자 context를 반영하는지 여부




## < 2.2 Existing Bandit Algorithms >

- bandit 문제의 핵심은 exploration과 exploitation의 균형임

### 1) 𝜖-greedy

- 확률 𝜖으로 무작위 arm 탐색, 1−𝜖로 현재 최선의 arm 선택
- 구조는 단순하지만 탐색이 guided되지 않아 비효율적일 수 있음
- long-term regret이 수렴하긴 하지만 속도는 느림

### 2) UCB (Upper Confidence Bound)

- 평균 보상 \( \hat{\mu}_{t,a} \)에 신뢰 구간을 더해 선택함
- 선택 기준: \( a_t = \arg\max_a \left( \hat{\mu}_{t,a} + c_{t,a} \right) \)
- 불확실성이 높은 arm을 자연스럽게 더 자주 선택함
- log(T) 수준의 regret을 보장함
- 대표적인 context-free 성능 보장 알고리즘임

### 3) EXP4

- 전문가(policy)들의 예측을 조합해 arm을 확률적으로 선택함  
- regret은 \( O(\sqrt{T \log N}) \) 수준으로 매우 우수함  
- 다만 전문가 수가 많아질수록 매 시점마다 가중치 업데이트 계산량이 커지고,  
  feature나 policy 공간이 클 경우 현실적으로 비효율적일 수 있음

### 4) Epoch-Greedy

- 일정 기간(epoch) 동안은 탐색만 수행, 이후에는 이용만 수행
- 구조는 간단하지만 regret이 \( O(T^{2/3}) \)로 큼

### 5) LinRel (LinUCB 기반 전신)

- 보상이 문맥 벡터의 선형결합이라고 가정
- 선형 회귀 기반 예측값 + UCB 구조를 결합함
- regret이 \( O(\sqrt{T}) \) 수준으로 좋고, 실제 구현도 비교적 쉬움

### 6) Bayesian Approaches

- Gittins Index, Thompson Sampling 등
- arm 보상에 대한 사전분포를 가정하고, posterior 기반으로 선택
- 이론적으로는 가장 최적에 가까운 방법이나, 계산량이 크고 사전정보가 필요한 점이 단점임



## < 정리 >

- context-free bandit은 arm 자체의 평균 보상만으로 선택 전략을 구성함
- 반면 contextual bandit은 사용자 및 arm의 feature를 문맥으로 활용함
- 다양한 기존 알고리즘은 exploration 방식, 계산 복잡도, regret 성능 측면에서 서로 trade-off가 존재함  
- 단순한 탐색 기반 알고리즘은 regret이 크고 수렴이 느리며,  
  계산량이 많은 알고리즘은 이론적 성능은 우수하지만 현실 적용이 어려운 경우가 많음  
- 이 논문은 그 중에서도 LinRel 구조를 기반으로,  
  계산 효율성과 regret 성능을 모두 고려한 LinUCB 알고리즘을 제안함  